In [1]:
#importing required libraries
from pyspark import SparkContext
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
#training: load spam and ham examples (two bags)
spam = sc.textFile("s3://tanush4/input/spam.txt")
ham = sc.textFile("s3://tanush4/input/ham.txt")
#initiate a feature vector
tf = HashingTF()
#each email is split into words, each word is mapped to one feature. Each email is mapped to a feature vector
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
hamFeatures = ham.map(lambda email: tf.transform(email.split(" ")))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1637604874581_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#for ML purpose (classifier), spam features are labeled as positive (1)
#non-spam features are labeled as negative (0)
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features)) 
negativeExamples = hamFeatures.map(lambda features: LabeledPoint(0, features)) #union spam and ham examples with labels now, for classifier
training_data = positiveExamples.union(negativeExamples)
#cache data since Logistic Regression is an iterative algorithm training_data.cache()
#use ML model logistic regression with SGD method
model = LogisticRegressionWithSGD.train(training_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#testing: first transform test cases to words (features)
posTestExample = tf.transform("O M G GET cheap stuff by sending money to ...".split(" ")) 
negTestExample = tf.transform("Hi Dad, I started studying Spark the other ...".split(" ")) #use classifier from training stage to predict the test cases
print ("Prediction for positive test example: %g" % model.predict(posTestExample))
print ("Prediction for negative test example: %g" % model.predict(negTestExample))
#call stop function from SparkContext to clean up resources
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Prediction for positive test example: 1
Prediction for negative test example: 0